In [1]:
import pandas as pd
import numpy as np
import time

In [16]:
test.head(20)

,ip,app,device,os,channel,day,hour,timestamp,minute,second,intesthh
0,5744,9,1,3,107,10,4,1510286400,0,0,1
1,119901,9,1,3,466,10,4,1510286400,0,0,1
2,72287,21,1,19,128,10,4,1510286400,0,0,1
3,78477,15,1,13,111,10,4,1510286400,0,0,1
4,123080,12,1,13,328,10,4,1510286400,0,0,1
5,110769,18,1,13,107,10,4,1510286400,0,0,1
6,12540,3,1,1,137,10,4,1510286400,0,0,1
7,88637,27,1,19,153,10,4,1510286400,0,0,1
8,14932,18,1,10,107,10,4,1510286400,0,0,1
9,123701,12,1,53,424,10,4,1510286400,0,0,1


In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

In [3]:
feature_count =  ['ip_day_intesthh_count',
                    'ip_day_hour_count',
                    'ip_os_day_hour_count',
                    'ip_app_day_hour_count',
                    'ip_app_os_day_hour_count',
                    'app_day_hour_count',
                    'ip_device_os_count',
                    'ip_app_device_os_count']

feature_mean = ['ip_device_os_mean',
                'ip_app_device_os_mean']

feature_time2nextclick = ['ip_device_os_time2previousclick',
                            'ip_app_device_os_time2previousclick']

feature_time2previousclick = ['ip_device_os_time2previousclick', 
                                'ip_app_device_os_time2previousclick']
    
    
feature_countfromfuture = ['ip_device_os_countfromfuture',
                            'ip_app_device_os_countfromfuture']

feature_countfrompast = ['ip_device_os_countfrompast',
                            'ip_app_device_os_countfrompast']
    
feature_lasttimediff =  ['ip_device_os_lasttimediff',
                             'ip_app_device_os_lasttimediff']

feature_ori = ['app', 'channel', 'device', 'os', 'hour']

feature_cols = []
added_feature = []

added_feature.extend(feature_count)
added_feature.extend(feature_mean)
added_feature.extend(feature_time2nextclick)
added_feature.extend(feature_time2previousclick)
added_feature.extend(feature_countfromfuture)
added_feature.extend(feature_countfrompast)
added_feature.extend(feature_lasttimediff)
feature_cols.extend(added_feature)
feature_cols.extend(feature_ori)

In [29]:
LEN = 18790469
LENG_TRAIN = 184903890
LEN_DAY9 = 53016937
OVERLAP = int(LEN *3 - LEN_DAY9) /2 
SHIFT = int(LEN - OVERLAP)
START0 = LENG_TRAIN - LEN_DAY9
START1 = START0 + SHIFT
START2 = START1 + SHIFT

In [33]:
train0 = train.iloc[START0:START0+LEN]
train1 = train.iloc[START1:START1+LEN]
train2 = train.iloc[START2:START2+LEN]

In [70]:
day = 9
seg = {}
seg['4start'] = [train[(train.day==day) &(train.hour==4)].index.values[0] for day in [7,8,9]]
seg['6end0sec'] = [train[(train.day==day) &(train.hour==6)&(train.second==1)].index.values[0] for day in [7,8,9]]
seg['6start'] = [train[(train.day==day) &(train.hour==6)].index.values[0] for day in [7,8,9]]
seg['9start'] = [train[(train.day==day) &(train.hour==9)].index.values[0] for day in [7,8,9]]
seg['11end0sec'] = [train[(train.day==day) &(train.hour==11)&(train.second==1)].index.values[0] for day in [7,8,9]]
seg['11start'] = [train[(train.day==day) &(train.hour==11)].index.values[0] for day in [7,8,9]]
seg['13start'] = [train[(train.day==day) &(train.hour==13)].index.values[0] for day in [7,8,9]]
seg['15end0sec'] = [train[(train.day==day) &(train.hour==15)&(train.second==1)].index.values[0] for day in [7,8,9]]
seg['15start'] = [train[(train.day==day) &(train.hour==15)].index.values[0] for day in [7,8,9]]

seg_test = {'test':{'4start':test[ (test.hour==4)].index.values[0] , 
              '6start':test[ (test.hour==6)].index.values[0] ,
              '6end0sec':test[ (test.hour==9)].index.values[0] , 
              '9start':test[ (test.hour==9)].index.values[0] , 
              '11start':test[ (test.hour==11)].index.values[0] ,
              '11end0sec':test[ (test.hour==13)].index.values[0] , 
              '13start':test[ (test.hour==13)].index.values[0] , 
              '15start':test[ (test.hour==15)].index.values[0] ,
              '15end0sec':len(test),
                   }}
df_test_re = pd.DataFrame(seg_test).transpose()
df_re = pd.DataFrame(seg, index=['day7', 'day8', 'day9'])
df_all = pd.concat([df_re, df_test_re])

df_all['seg1_count'] = df_all['6end0sec'] - df_all['4start']
df_all['seg2_count'] = df_all['11end0sec'] - df_all['9start']
df_all['seg3_count'] = df_all['15end0sec'] - df_all['13start']
df_all['total_count'] = df_all['seg1_count'] + df_all['seg2_count'] + df_all['seg3_count']

In [105]:
df_all.

,11end0sec,11start,13start,15end0sec,15start,4start,6end0sec,6start,9start,seg1_count,seg2_count,seg3_count,total_count
day7,44315465,44314504,50448795,56846651,56845831,22536987,29410665,29409829,38052439,6873678,6263026,6397856,19534560
day8,104525803,104524970,111441304,118735620,118734633,82259195,88965389,88964596,98079570,6706194,6446233,7294316,20446743
day9,168265844,168264868,174976526,181878212,181877332,144708152,152413509,152412584,161974465,7705357,6291379,6901686,20898422
test,12316147,12315734,12316147,18790469,18789970,0,6202933,6202552,6202933,6202933,6113214,6474322,18790469


In [108]:
df_all.to_csv(path +'hourdistri.csv')